#### 处理录音srt

In [7]:
from pathlib import Path
import pandas as pd
import srt

找到路径下所有录播wav片段

In [8]:
file_list = [i for i in Path(r'C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\remove\demucs_renamed').rglob("*.wav")]
file_list

[WindowsPath('C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/remove/demucs_renamed/1_【直播回放】11 2022年11月10日20点场 - 1.【直播回放】11 2022年11月10日20点场(Av817466437,P1)_01_(Vocals).wav'),
 WindowsPath('C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/remove/demucs_renamed/2_【直播回放】11 2022年11月10日20点场 - 1.【直播回放】11 2022年11月10日20点场(Av817466437,P1)_02_(Vocals).wav'),
 WindowsPath('C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/remove/demucs_renamed/3_【直播回放】11 2022年11月10日20点场 - 1.【直播回放】11 2022年11月10日20点场(Av817466437,P1)_03_(Vocals).wav'),
 WindowsPath('C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/remove/demucs_renamed/4_【直播回放】11 2022年11月10日20点场 - 1.【直播回放】11 2022年11月10日20点场(Av817466437,P1)_04_(Vocals).wav'),
 WindowsPath('C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/remove/demucs_renamed/5_【直播回放】11 2022年11月10日20点场 - 1.【直播回放】11 2022年11月10日20点场(Av817466437,P1)_05_(Vocals).wav'),
 WindowsPath('C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/remove/demucs_r

重命名并保存到新目录

In [5]:
import shutil

new_dir = r'C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\remove\demucs_renamed\renamed'

for file in file_list:
    new_name = '20221110-' + Path(file).stem.split('_')[-2] + '.wav'
    shutil.copy2(file, Path(new_dir) / Path(new_name))
    
file_list = [i for i in Path(new_dir).rglob('*')]


读取字幕列表，重命名并保存到新目录

In [9]:

srt_list = [i for i in Path(r'C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\remove\demucs_renamed').rglob('*.srt')]

for file in srt_list:
    new_name = '20221110-' + Path(file).stem.split('_')[-2] + '.srt'
    shutil.copy2(file, Path(new_dir) / Path(new_name))

新录音，字幕组合

In [9]:
new_dir = r'C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\remove\demucs_renamed\renamed'
srt_list = [i for i in Path(new_dir).rglob('*.srt')]
wav_list = [i for i in Path(new_dir).rglob('*.wav')]
key_list = [i.stem for i in wav_list]
df = pd.DataFrame([key_list, wav_list, srt_list]).T
df.columns = ['key', 'wav', 'srt']
df

,key,wav,srt
0,20221110-01,C:\Users\63537\Desktop\projects\TTS\data\audio...,C:\Users\63537\Desktop\projects\TTS\data\audio...
1,20221110-02,C:\Users\63537\Desktop\projects\TTS\data\audio...,C:\Users\63537\Desktop\projects\TTS\data\audio...
2,20221110-03,C:\Users\63537\Desktop\projects\TTS\data\audio...,C:\Users\63537\Desktop\projects\TTS\data\audio...
3,20221110-04,C:\Users\63537\Desktop\projects\TTS\data\audio...,C:\Users\63537\Desktop\projects\TTS\data\audio...
4,20221110-05,C:\Users\63537\Desktop\projects\TTS\data\audio...,C:\Users\63537\Desktop\projects\TTS\data\audio...
5,20221110-06,C:\Users\63537\Desktop\projects\TTS\data\audio...,C:\Users\63537\Desktop\projects\TTS\data\audio...


In [10]:
from tqdm import tqdm
from pydub import AudioSegment

cuts_dir = r'C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\cuts\wav'
res_save_path = r'C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\cuts\20221110-all.txt'

with open(res_save_path,'w',encoding='utf8') as res_file:

    for row in df.itertuples():
        with open(row.srt,'r', encoding='utf-8') as f:
            srt_text = f.read()
        srt_objs = [i for i in srt.parse(srt_text)]

        audio:AudioSegment = AudioSegment.from_file(row.wav)
        if audio.channels == 2:
            audio = audio.split_to_mono()[0]
        for i in tqdm(srt_objs):
            # 开始结束
            start_milli_sec = int(i.start.total_seconds()*1000)
            end_milli_sec = int(i.end.total_seconds()*1000)
            # 录音段导出
            out_name = f'{row.key}-{start_milli_sec}-{end_milli_sec}.wav'
            out_path = Path(cuts_dir) /  Path(out_name)
            cut = audio[start_milli_sec:end_milli_sec]
            cut.export(out_path, format='wav')
            # 结果保存
            text = i.content
            res_file.write(f'{out_path}|yua|{text}\n')

    

100%|██████████| 196/196 [00:00<00:00, 1370.60it/s]


### 提取 emotion embeddings

In [1]:
import torch
import torch.nn as nn
from transformers import Wav2Vec2Processor
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2Model,
    Wav2Vec2PreTrainedModel,
)
import os
import librosa
import numpy as np

import sys
sys.path.append('..')
from emotion_extract import EmotionModel, torch_extract

c:\Users\63537\anaconda3\envs\emotional-vits\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load model from hub   
device = 'cuda' if torch.cuda.is_available() else "cpu"
model_name = '../emotion-model/wav2vec2-large-robust-12-ft-emotion-msp-dim'
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = EmotionModel.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import torchaudio

file_lists = [Path(r'C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/cuts/20221110-all.txt')]

emb_list = []
for filelist in file_lists:
    print(filelist, "----start emotion extract-------")
    df = pd.read_table(filelist, sep='|',header=None)

    for file in tqdm(df[0]):
        wav, sr = torchaudio.load(file)
        # if wav.dim() >= 2:
        #     wav = wav.squeeze(0)
        wav = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16_000)(wav)
        if len(wav[0]) < (16 * 500): # 最小 500ms
            continue
        emb = torch_extract(x=wav, sampling_rate=16_000, embeddings=True, 
                            processor=processor, model=model, device=device)
        emb_list.append(
            {'emb_name': file + '.emo.npy', 'emb':emb}
        )

C:\Users\63537\Desktop\projects\TTS\data\audio\20221110\cuts\20221110-all.txt ----start emotion extract-------


100%|██████████| 2028/2028 [00:47<00:00, 42.34it/s]


In [22]:
print(emb_list[0])
for d in tqdm(emb_list):
    np.save(f"{d['emb_name']}", d['emb'].squeeze(0))

{'emb_name': 'C:\\Users\\63537\\Desktop\\projects\\TTS\\data\\audio\\20221110\\cuts\\wav\\20221110-01-966-1568.wav.emo.npy', 'emb': array([[-0.00760905,  0.00558934, -0.0115053 , ...,  0.0071029 ,
         0.00709296,  0.00472145]], dtype=float32)}


100%|██████████| 1815/1815 [00:01<00:00, 1594.90it/s]


In [32]:
import pandas as pd
from pydub import AudioSegment

df = pd.read_table(filelist, sep='|',header=None)
df['length'] = df[0].apply(lambda x: len(AudioSegment.from_file(x)))
df = df[df['length'] >= 500]
df.columns = ['wav','spk','text','length']
df['emo-emb'] = df['wav'].apply(lambda x: Path(x).parent / Path(Path(x).name + '.emo.npy'))
df.to_csv(Path(r'C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/cuts/20221110-all-emo-above0.5.csv'), index=None)

In [31]:
Path(r'C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/cuts/20221110-all-emo-above0.5.csv').parent


WindowsPath('C:/Users/63537/Desktop/projects/TTS/data/audio/20221110/cuts')

In [13]:
y = processor(wav, sampling_rate=16000)
y

{'input_values': [array([[-0.01334807, -0.02073031, -0.01764838, ...,  0.00980827,
         0.01093164,  0.00634813]], dtype=float32)], 'attention_mask': [array([1])]}

In [20]:
model(torch.from_numpy(y['input_values'][0]).to(device))[0].detach()

tensor([[-0.0073,  0.0060, -0.0060,  ...,  0.0074,  0.0078,  0.0053]],
       device='cuda:0')